In [1]:
from PIL import Image
import numpy as np
import tensorflow as tf
import os.path
import glob

import math
import h5py
import matplotlib.pyplot as plt
import scipy
from scipy import ndimage
from tensorflow.python.framework import ops
from cnn_utils import *

%matplotlib inline
np.random.seed(1)

In [2]:
#resize images in a folder to 64 by 64
def image_resize(folder):
    images = glob.glob("img/" + folder + "/*.jpg")
    for image in images:
        im_path = os.path.abspath(image)
        im = Image.open(im_path)
        im = im.resize((64, 64), Image.ANTIALIAS)
        im.save(os.path.splitext(im_path)[0] + '-small.jpg')

In [11]:
#convert training and test set images to tensors
def convert_to_tensor(folder):
    x = np.empty((0, 64, 64, 3))
    y = np.empty((0, 2))
    directory_name = "img/" + folder
    for filename in os.listdir(directory_name):
        if filename.endswith("-small.jpg"):
            img = Image.open(os.path.join(directory_name, filename)).convert('RGB')
            x = np.append(x, np.array(img).reshape((1, 64, 64, 3)), axis = 0)
            if filename.startswith('owl'):
                y = np.append(y, np.array([1, 0]).reshape((1, 2)), axis = 0)
                print(filename)
                print(y[-1])
            else: 
                y = np.append(y, np.array([0, 1]).reshape((1, 2)), axis = 0)
                print(filename)
                print(y[-1])
    return [x, y]

In [13]:
#resizing manual test images in "manual" folder
image_resize("manual")

#convert to tensor
xmanual, ymanual = convert_to_tensor("manual")

print(xmanual.shape)
print(ymanual.shape)

ball20-small.jpg
[0. 1.]
ball21-small.jpg
[0. 1.]
owl20-small.jpg
[1. 0.]
owl21-small.jpg
[1. 0.]
(4, 64, 64, 3)
(4, 2)


In [5]:
# interactive session can avoid sess.run()
sess = tf.InteractiveSession()

In [6]:
#load and restore model
new_saver = tf.train.import_meta_graph('model.ckpt.meta')
new_saver.restore(sess, 'model.ckpt')

INFO:tensorflow:Restoring parameters from model.ckpt


In [7]:
# quickly glance at node values
tf.get_default_graph().as_graph_def()

node {
  name: "input"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
        dim {
          size: 64
        }
        dim {
          size: 64
        }
        dim {
          size: 3
        }
      }
    }
  }
}
node {
  name: "Placeholder"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
        dim {
          size: 2
        }
      }
    }
  }
}
node {
  name: "W1/Initializer/random_uniform/shape"
  op: "Const"
  attr {
    key: "_class"
    value {
      list {
        s: "loc:@W1"
      }
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
          dim {
            size: 4
       

In [8]:
# get input and output
x = sess.graph.get_tensor_by_name("input:0")
y_conv = sess.graph.get_tensor_by_name("output:0")

In [9]:
print(x)
print(y_conv)

Tensor("input:0", shape=(?, 64, 64, 3), dtype=float32)
Tensor("output:0", shape=(?,), dtype=int64)


In [14]:
# prediction to unseen dataset
result = sess.run(y_conv, feed_dict={x:xmanual})
print(result)

[1 1 0 0]
